# File to fill the test.csv for prediction

In [1]:

import numpy as np
import sys
import pandas as pd
from tqdm import tqdm
import shutil

sys.path.append("../")
from detection import YoloInference
from notebooks.utils_notebook import filter_boxes

In [ ]:
itos={0:'B', 1:'BA', 2:'EO', 3:'Er', 4:'LAM3', 5:'LF', 6:'LGL', 7:'LH_lyAct', 8:'LLC', 9:'LM', 10:'LY', 11:'LZMG', 12:'LyB', 13:'Lysee', 14:'M', 15:'MBL', 16:'MM', 17:'MO', 18:'MoB', 19:'PM', 20:'PNN', 21:'SS', 22:'Thromb'}


data_path="../data/Cytologia/images"


test_csv_path="../data/Cytologia/test.csv"
csv_path="../data/Cytologia/predictions.csv"
shutil.copy(test_csv_path,"../data/Cytologia/predictions.csv")

df = pd.read_csv(csv_path)
yolo_engine=YoloInference("../models/detection/Cytologia_msk_iou_cheat_0.8/yolo11n/384/curated250/train/weights/best.pt",device="cuda")
#yolo_engine=YoloInference("models/detection/Cytologia_msk_iou/yolo11m/384/curated250/train/weights/best.pt",device="cuda") #! BASELINE
#yolo_engine=YoloInference("models/detection/Cytologia_yolo_msk_black/yolov10n/384/curated/train/weights/best.pt",device="cuda")

# Get unique NAME to infer each image only once
if not {'x1', 'y1', 'x2', 'y2', 'class'}.issubset(df.columns):
    for col in ['x1', 'y1', 'x2', 'y2', 'class']:
        df[col] = np.nan

# Get unique NAME
names = df["NAME"].unique()
tqdm_names= tqdm(names)
no_dets=0
for name in tqdm_names:
    img_path=f"{data_path}/{name}"
    # get occurences of name
    occurences = df[df["NAME"]==name]
    trustii_ids = occurences["trustii_id"].tolist()
    boxes,scores,labels=yolo_engine.predict(img_path,verbose=False,conf=0.00001,return_probs=False)

    boxes=list(boxes)
    scores=list(scores)
    labels=list(labels)
    
    if len(occurences) < len(boxes):
        boxes,scores,labels=filter_boxes(occurences,boxes,scores,labels)

    most_conf_cls=None
    for idx,(box,score,label) in enumerate(zip(boxes,scores,labels)):
        if idx==0:
            most_conf_cls=itos[label]
        x1,y1,x2,y2 = box
        trustii_id = trustii_ids[idx]
        cls=itos[label] #if score>0.1 else most_conf_cls # ! seems to work bad ....
        df.loc[df["trustii_id"] == trustii_id, ["x1", "y1", "x2", "y2", "class"]] = [x1, y1, x2, y2, cls]

    # case where a wbc is not detected (even with this very low threshold)

    for i in range(len(boxes),len(occurences)):
        trustii_id = trustii_ids[i]
        # better if we take mean of boxes along the full dataset ? (depending if there is other cells in the image)
        # we can also call the classifier on the full image (another classifier)
        df.loc[df["trustii_id"] == trustii_id, ["x1", "y1", "x2", "y2", "class"]] = [0, 0, 0, 0, 'PNN']

print("Number of undetected WBCs: ",no_dets)
df.to_csv(csv_path, index=False)
print("CSV mis à jour avec succès.")
